## Anderson Silva's xxx WQ-MAKER genome annotation

In [1]:
# After mounting the volume, change to that volume
cd /vol_c

In [2]:
# Check the contents of the volume
ls

lost+found  moss_WQ_MAKER


In [3]:
# Copy the contents from Datastore
iget -rPVT /iplant/home/andersonsi/Shared_upendra/New_WQ_MAKER

NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
NOTICE: created irodsHome=/iplant/home/upendra_35
NOTICE: created irodsCwd=/iplant/home/upendra_35
D- ./New_WQ_MAKER :
0/4 -  0.00% of files done   0.000/414.854 MB -  0.00% of file sizes done
Processing My_CDS_min_len150_1.fasta - 31.075 MB   2017-10-22.18:58:23
   My_CDS_min_len150_1.fasta      31.075 MB | 2.260 sec | 0 thr | 13.749 MB/s
1/4 - 25.00% of files done   31.075/414.854 MB -  7.49% of file 

In [4]:
# Check the contents again
ls

lost+found  moss_WQ_MAKER  New_WQ_MAKER


In [5]:
# Change the name of the folder
mv New_WQ_MAKER moss_WQ_MAKER

In [6]:
# Change to the new folder
cd moss_WQ_MAKER

In [7]:
# Check the contents again
ls

final_assembly.gapclosed.fa  My_Final_Unigene.fasta
My_CDS_min_len150_1.fasta    My_Pep_min_len50.fasta


In [8]:
# Make a new directory and move the contents to the new directory
mkdir data && mv * data

mv: cannot move 'data' to a subdirectory of itself, 'data/data'


: 1

In [9]:
# Check the contents again
ls

data


In [10]:
# Check the contents in the data folder
ls data

final_assembly.gapclosed.fa  My_Final_Unigene.fasta
My_CDS_min_len150_1.fasta    My_Pep_min_len50.fasta


In [11]:
# Navigate to the data folder
cd data

In [12]:
# Combine the 2 transcriptome/est files 
cat My_CDS_min_len150_1.fasta My_Final_Unigene.fasta > My_CDS_min_len150_1_Unigene_comb.fasta

In [13]:
# Navigate one directory up
cd ..

## Set-up MAKER run

In [14]:
# Create maker files
maker -CTL

Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.


In [15]:
# Remove the maker_opts.ctl file
rm maker_opts.ctl

In [16]:
# Copy the maker_opts.ctl file from the example files
cp /opt/WQ-MAKER_example_data/maker_opts.ctl .

In [17]:
# After making changes the maker_opts.ctl file, check to see if the new data exits
grep "data/" maker_opts.ctl

genome=./data/final_assembly.gapclosed.fa  #genome sequence (fasta file or fasta embeded in GFF3 file)
est=./data/My_CDS_min_len150_1_Unigene_comb.fasta #set of ESTs or assembled mRNA-seq in fasta format
protein=./data/My_Pep_min_len50.fasta  #protein sequence file in fasta format (i.e. from mutiple oransisms)
snoscan_rrna=./test_data/Os-rRNA.fa #rRNA file to have Snoscan find snoRNAs


## Ansible set-up

In [18]:
# Copy the ansible config file from the example folder
cp /opt/WQ-MAKER_example_data/.ansible.cfg ~

In [19]:
# Display the content of the ansible config file
cat /home/upendra/.ansible.cfg

[defaults]
host_key_checking = False


In [32]:
# Copy the maker-hosts file from the example folder
cp /opt/WQ-MAKER_example_data/maker-hosts .

In [33]:
# Append the ip address of your workers to the maker-hosts
echo "149.165.169.21" >> maker-hosts
echo "149.165.169.78" >> maker-hosts
echo "129.114.104.35" >> maker-hosts
echo "129.114.104.100" >> maker-hosts
echo "129.114.104.61" >> maker-hosts
echo "129.114.104.138" >> maker-hosts
echo "129.114.104.136" >> maker-hosts
echo "129.114.104.104" >> maker-hosts
echo "129.114.104.141" >> maker-hosts
echo "129.114.104.130" >> maker-hosts

In [34]:
# Display the contents of the maker-hosts file
cat maker-hosts

[workers]
149.165.169.21
149.165.169.78
129.114.104.35
129.114.104.100
129.114.104.61
129.114.104.138
129.114.104.136
129.114.104.104
129.114.104.141
129.114.104.130


In [35]:
# Copy the Ansible playbook from the example folder
cp /opt/WQ-MAKER_example_data/worker-launch.yml .

In [36]:
# Display the contents of the ansible playbook
cat worker-launch.yml

---
- hosts : workers
  environment:
    PATH: "{{ ansible_env.PATH }}:/home/${USER}/bin:/home/${USER}/.local/bin:/opt/icommands:/opt/icommands:/opt/exonerate-2.2.0-x86_64/bin/:/opt/cctools/bin:/opt/ncbi-blast-2.6.0+/bin/:/opt/snoscan-0.9.1/:/opt/tRNAscan-SE-1.3.1/:/opt/snap/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/opt/augustus-3.2.2/bin:/opt/maker/bin:/opt/RepeatMasker:/opt/snap"
    PERL5LIB: "/opt/tRNAscan-SE-1.3.1::/opt/cctools/lib/perl5/site_perl"
  tasks :
  - name : Execute the script
    shell : /opt/cctools/bin/work_queue_worker -N wq_test_${USER} -s /home/${USER} --cores all --debug-rotate-max=0 -d all -o /home/${USER}/worker.dbg


In [37]:
# Change the job name in the ansible playbook
sed 's/wq_test_${USER}/wq_mossa_${USER}/g' worker-launch.yml > temp && mv temp worker-launch.yml

In [38]:
# Display the contents of the ansible playbook
cat worker-launch.yml

---
- hosts : workers
  environment:
    PATH: "{{ ansible_env.PATH }}:/home/${USER}/bin:/home/${USER}/.local/bin:/opt/icommands:/opt/icommands:/opt/exonerate-2.2.0-x86_64/bin/:/opt/cctools/bin:/opt/ncbi-blast-2.6.0+/bin/:/opt/snoscan-0.9.1/:/opt/tRNAscan-SE-1.3.1/:/opt/snap/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/opt/augustus-3.2.2/bin:/opt/maker/bin:/opt/RepeatMasker:/opt/snap"
    PERL5LIB: "/opt/tRNAscan-SE-1.3.1::/opt/cctools/lib/perl5/site_perl"
  tasks :
  - name : Execute the script
    shell : /opt/cctools/bin/work_queue_worker -N wq_mossa_${USER} -s /home/${USER} --cores all --debug-rotate-max=0 -d all -o /home/${USER}/worker.dbg


In [39]:
# Run work queue maker on the MASTER
wq_maker -port 9123 -contigs-per-split 1 -cores 1 -memory 4296 -disk 8392 -N wq_mossa_${USER} -d all -o master.dbg -debug_size_limit=0 > log_file.txt 2>&1 &

[1] 16983


In [40]:
# Show tail of the log file
tail log_file.txt

Sun Oct 22 20:20:48 2017 :: Submitting file ./data/final_assembly.gapclosed.fa_001998 for processing.
Sun Oct 22 20:20:48 2017 :: Submitted task 2676 for annotating ./data/final_assembly.gapclosed.fa_001998 with command:  maker -g ./data/final_assembly.gapclosed.fa_001998 -base final_assembly.gapclosed -debug_size_limit=0
Sun Oct 22 20:20:48 2017 :: Submitting file ./data/final_assembly.gapclosed.fa_000727 for processing.
Sun Oct 22 20:20:48 2017 :: Submitted task 2677 for annotating ./data/final_assembly.gapclosed.fa_000727 with command:  maker -g ./data/final_assembly.gapclosed.fa_000727 -base final_assembly.gapclosed -debug_size_limit=0
Sun Oct 22 20:20:48 2017 :: Submitting file ./data/final_assembly.gapclosed.fa_000800 for processing.
Sun Oct 22 20:20:48 2017 :: Submitted task 2678 for annotating ./data/final_assembly.gapclosed.fa_000800 with command:  maker -g ./data/final_assembly.gapclosed.fa_000800 -base final_assembly.gapclosed -debug_size_limit=0
Sun Oct 22 20:20:48 2017 :: 

In [41]:
# Run ansible playbook to run work queue maker on the workers
ansible-playbook -u ${USER} -i maker-hosts worker-launch.yml > log_file_2.txt 2>&1 &

[2] 17190


In [42]:
# Check the status of work queue maker
work_queue_status -M wq_mossa_${USER}

PROJECT            HOST                   PORT WAITING RUNNING COMPLETE WORKERS 
wq_mossa_upendra   js-129-114-104-169.je  9123    2616      14       49       5 


In [43]:
# For some reason, the Ansible can only run 5 remote servers at time. So I have to copy the maker-hosts file
cp /opt/WQ-MAKER_example_data/maker-hosts maker-hosts2

In [44]:
# Append the ip address of your workers to the maker-hosts2 file
echo "129.114.104.138" >> maker-hosts2
echo "129.114.104.136" >> maker-hosts2
echo "129.114.104.104" >> maker-hosts2
echo "129.114.104.130" >> maker-hosts2
echo "129.114.104.141" >> maker-hosts2

In [45]:
# Run ansible playbook to run work queue maker on the workers using maker-host2 file
ansible-playbook -u ${USER} -i maker-hosts2 worker-launch.yml > log_file_3.txt 2>&1 &

[3] 17649


In [52]:
# Check the tail of the log file
tail log_file.txt

WQ-MAKER End_time:	1508753005000000
WQ-MAKER Elapsed:	0d 8:43:32.000000
Work Queue Wall Time:	0d 8:42:36.640545
Cumulative Task Wall Time:	3d 14:43:46.772623
Cumulative Task Good Execute Time:	3d 14:43:46.772623
Work Queue Send Time:	0d 0:07:43.423778
Work Queue Receive Time:	0d 0:21:02.178819
-----------------------------------------------------------------
Mon Oct 23 05:03:25 2017 :: MPI used :: Cores 1 :: Memory 4296 :: Disk 8392 
-----------------------------------------------------------------


In [53]:
# Navigate to the output folder
cd final_assembly.gapclosed.maker.output

In [54]:
# Check the contents of the log file
head final_assembly.gapclosed_master_datastore_index.log

ScPhAot_1_HRSCAF_2	final_assembly.gapclosed_datastore/C7/FA/ScPhAot_1_HRSCAF_2/	STARTED
ScPhAot_2_HRSCAF_3	final_assembly.gapclosed_datastore/7E/E0/ScPhAot_2_HRSCAF_3/	STARTED
ScPhAot_3_HRSCAF_4	final_assembly.gapclosed_datastore/1B/86/ScPhAot_3_HRSCAF_4/	STARTED
ScPhAot_4_HRSCAF_5	final_assembly.gapclosed_datastore/0D/36/ScPhAot_4_HRSCAF_5/	STARTED
ScPhAot_5_HRSCAF_6	final_assembly.gapclosed_datastore/B5/E9/ScPhAot_5_HRSCAF_6/	STARTED
ScPhAot_6_HRSCAF_7	final_assembly.gapclosed_datastore/40/09/ScPhAot_6_HRSCAF_7/	STARTED
ScPhAot_7_HRSCAF_10	final_assembly.gapclosed_datastore/21/EA/ScPhAot_7_HRSCAF_10/	STARTED
ScPhAot_8_HRSCAF_11	final_assembly.gapclosed_datastore/54/3E/ScPhAot_8_HRSCAF_11/	STARTED
ScPhAot_9_HRSCAF_12	final_assembly.gapclosed_datastore/FD/B6/ScPhAot_9_HRSCAF_12/	STARTED
ScPhAot_10_HRSCAF_13	final_assembly.gapclosed_datastore/D1/47/ScPhAot_10_HRSCAF_13/	STARTED


In [55]:
# Check the contents of the log file
tail final_assembly.gapclosed_master_datastore_index.log

ScPhAot_2670_HRSCAF_3033	final_assembly.gapclosed_datastore/09/09/ScPhAot_2670_HRSCAF_3033/	FINISHED
ScPhAot_2671_HRSCAF_3034	final_assembly.gapclosed_datastore/32/38/ScPhAot_2671_HRSCAF_3034/	FINISHED
ScPhAot_2672_HRSCAF_3035	final_assembly.gapclosed_datastore/D8/6E/ScPhAot_2672_HRSCAF_3035/	FINISHED
ScPhAot_2673_HRSCAF_3036	final_assembly.gapclosed_datastore/84/93/ScPhAot_2673_HRSCAF_3036/	FINISHED
ScPhAot_2674_HRSCAF_3037	final_assembly.gapclosed_datastore/54/96/ScPhAot_2674_HRSCAF_3037/	FINISHED
ScPhAot_2675_HRSCAF_3038	final_assembly.gapclosed_datastore/78/B9/ScPhAot_2675_HRSCAF_3038/	FINISHED
ScPhAot_2676_HRSCAF_3040	final_assembly.gapclosed_datastore/44/F5/ScPhAot_2676_HRSCAF_3040/	FINISHED
ScPhAot_2677_HRSCAF_3041	final_assembly.gapclosed_datastore/0E/E2/ScPhAot_2677_HRSCAF_3041/	FINISHED
ScPhAot_2678_HRSCAF_3042	final_assembly.gapclosed_datastore/75/B9/ScPhAot_2678_HRSCAF_3042/	FINISHED
ScPhAot_2679_HRSCAF_3116	final_assembly.gapclosed_datastore/56/36/ScPhAot_2679_HRSCAF_3116/

In [56]:
# Merge the gff files
gff3_merge -n -d final_assembly.gapclosed_master_datastore_index.log

In [57]:
# Check the contents of the output folder
ls

final_assembly.gapclosed.all.gff		     maker_exe.log
final_assembly.gapclosed_datastore		     maker_opts.log
final_assembly.gapclosed_master_datastore_index.log  mpi_blastdb
maker_bopts.log


In [63]:
# Check the number of contigs annotated
cut -f1 final_assembly.gapclosed.all.gff | sort | uniq | grep -v "#" | wc -l

2672


In [64]:
# Check the number of contigs int the fasta file
grep ">" -c ../data/final_assembly.gapclosed.fa

2679
